# Methods
- Class **TobiiTsvObject**
    - Takes a tsv file from the Tobii software
    - tobii = TobiiTsvObject('ALL-DATA.tsv')
    
- Methods

    1. getSaccadicData()
        * **Outputs information on the saccadic data in the form of a dictionary with the path Participants->Images->Data:**
        1. Amount of saccades for image for participant (int)
        2. Mean duration of saccades for image for participant (float)
        3. Mean saccade velocity for image for participant (float)
        4. Saccade velocity peak for image for participant (float)


In [6]:
class TobiiTsvObject:
    
    # Variables
    data = []
    headers = []
    
    #init
    def __init__(self, tsvPath):
        self.data, self.headers = self.importTsv(tsvPath)
    
    #Init Methods
    def importTsv(self, tsvPath):
        import csv
        tobiiData = []
        with open(tsvPath,'rb') as tsvin:
            tsvin = csv.reader(tsvin, delimiter='\t')
            count = 0
            for row in tsvin:
                tobiiData.append(row)
        tobiiHeaders = []
        count = 0
        for header in tobiiData[0]:
            tobiiHeaders.append(str(header))
            print str(count) +": " + header
            count += 1
        return [tobiiData, tobiiHeaders]
    
    #Methods
    # call AOIHitCount(HeaderIndex) to get a count of hits misses and total within the AOI
    def AOIHitCount(self, HeaderIndex):
        hit = []
        for data in self.data[1:]:
            hit.append(data[HeaderIndex])
        hits = hit.count("1")
        miss = hit.count("0")
        total = hits + miss
        return [hits, miss, total]
    
    def getSaccades(self, AOIList, time=500):
        saccades = []            
        # 45 46 47            
        inputAOIs = AOIList
        prevAOI = None
        prevAOITime = 0
        currentAOI = None
        for row in tobii.data[1:]:
            for AOI in AOIList:
                if row[AOI] == "1":
                    prevAOI = currentAOI
                    currentAOI = tobii.headers[AOI]
                    if currentAOI == prevAOI:
                        prevAOITime = int(row[31])
                    if (int(row[31]) - prevAOITime) < time and currentAOI != prevAOI and prevAOI != None:
                        #print prevAOI[4:-4]+"\t->\t"+currentAOI[4:-4]
                        saccades.append([prevAOI[4:-4], currentAOI[4:-4], int(row[31]) - prevAOITime])#prevAOITime, int(row[31])])
        return saccades

    def getParticipants(self):
        participants = []
        for row in tobii.data[1:]:
            if row[4] not in participants:
                participants.append(row[4])
        return participants
    
    def getMediaRowForParticipants(self, participantsIn):
        participants = {}
        for participant in participantsIn:
            media = {}
            index = 1
            for row in tobii.data[1:]:
                if participant == row[4] and row[18] != "":
                    if row[18] not in media:
                        media[row[18]] = [index]
                    else:
                        media[row[18]].append(index)
                index += 1
            media2 = {}
            for key in media.keys():
                first = media[key][0]
                last = media[key][-1]
                media2[key] = [first, last]
            participants[participant] = media2
        return participants
    
    def getTotalSaccades(self, saccadeList):
        return len(saccadeList)
    
    def getSaccadeDurationMean(self, durationDict):
        dictSum = sum(durationDict.values())
        dictLen = len(durationDict.values())
        return float(dictSum) / float(dictLen)
    
    def getSaccadeVelocityMean(self, velocityDict):
        velSum = sum(velocityDict.values())
        velLen = len(velocityDict.values())
        return float(velSum) / float(velLen)
    
    def getSaccadeVelocityPeak(self, velocityDict):
        return max(velocityDict.values())
    
    def stringToFloat(self, string):
        for idx in xrange(len(string)):
            if string[idx] == ".":
                return float(string[:idx]) + float(string[idx:])
        return "ERROR"
    
    def getSaccadicData(self):
        participants = {}
        xData = self.getMediaRowForParticipants(tobii.getParticipants())
        for participant in xData:
            participants[participant] = {}
            for image in xData[participant]:
                participants[participant][image] = []
                fromRow = xData[participant][image][0]
                toRow = xData[participant][image][1]
        
                indexTotal = 0
                durationMean = 0
                #durationPeak = 0
                velocityMean = 0
                velocityPeak = 0
        
                index = []
                duration = {}
                velocity = {}

                prevSacIdx = None
                currSacIdx = None
                sacDurStart = 0
                sacDurEnd = 0
                for row in tobii.data[fromRow:toRow]:
                    prevSacIdx = currSacIdx
                    currSacIdx = row[36]
                    if row[36] not in index and row[36] != "":
                        index.append(row[36])
            
                    
                    if currSacIdx != "" and currSacIdx!= None and prevSacIdx == "":
                        sacDurStart = row[31]
                    elif prevSacIdx != "" and currSacIdx == "":
                        sacDurEnd = row[31]
                        sacDur = int(sacDurEnd) - int(sacDurStart)
                        duration[prevSacIdx] = sacDur
        
                        if row[41] != "":
                            curAmp = str(row[41]).replace(",",".")
                            curAmp = self.stringToFloat(curAmp)
                            curDur = float(duration[prevSacIdx])/1000 #SECONDS
        
                            velocity[prevSacIdx] =  curAmp / curDur
            
                indexTotal = tobii.getTotalSaccades(index)
                durationMean = self.getSaccadeDurationMean(duration)
                velocityMean = self.getSaccadeVelocityMean(velocity)
                velocityPeak = self.getSaccadeVelocityPeak(velocity)
                participants[participant][image] = [indexTotal, durationMean, velocityMean, velocityPeak]
        return participants

In [7]:
tobii = TobiiTsvObject('ALL-DATA.tsv')
#tobii = TobiiTsvObject("mettemodel3bothglobal.tsv")

0: ﻿ExportDate
1: StudioVersionRec
2: StudioProjectName
3: StudioTestName
4: ParticipantName
5: [Q01]Value
6: [mq1]Value
7: [q2]Value
8: [q2_2]Value
9: [mq1_2]Value
10: [q2_1]Value
11: [mq1_1]Value
12: RecordingName
13: RecordingDate
14: RecordingDuration
15: RecordingResolution
16: PresentationSequence
17: FixationFilter
18: MediaName
19: MediaPosX (ADCSpx)
20: MediaPosY (ADCSpx)
21: MediaWidth
22: MediaHeight
23: SegmentName
24: SegmentStart
25: SegmentEnd
26: SegmentDuration
27: SceneName
28: SceneSegmentStart
29: SceneSegmentEnd
30: SceneSegmentDuration
31: RecordingTimestamp
32: LocalTimeStamp
33: EyeTrackerTimestamp
34: MouseEvent
35: FixationIndex
36: SaccadeIndex
37: GazeEventType
38: GazeEventDuration
39: FixationPointX (MCSpx)
40: FixationPointY (MCSpx)
41: SaccadicAmplitude
42: AbsoluteSaccadicDirection
43: RelativeSaccadicDirection
44: AOI[m1bMessy-AOI]Hit
45: AOI[m1bxnMessy]Hit
46: AOI[m1boMessy]Hit
47: AOI[m1bQuestion]Hit
48: AOI[m1aNotMesyQuestion]Hit
49: AOI[m1aNotMessy

In [8]:
print tobii.getSaccadicData()

{'P01': {'model4agq.png': [271, 404.1033210332103, 147.3972344782999, 2980.0], 'model1agq.png': [82, 910.6219512195122, 218.7807284440233, 4453.333333333333], 'model1bgq.png': [224, 67.12, 139.42686505646566, 1655.7142857142856], 'model4bgq.png': [335, 544.4791666666666, 128.52371959013118, 1581.0]}}


In [9]:
[271, 404.1033210332103, 147.3972344782999, 2980.0]

[271, 404.1033210332103, 147.3972344782999, 2980.0]

In [5]:
[335, 544.4791666666666, 128.52371959013118, 1581.0]

[335, 544.4791666666666, 128.52371959013118, 1581.0]